In [1]:
import random
import numpy as np

class card:

    def __init__(self, suit, value):
        self.suit = suit
        self.value = value
        self.values = {14:"Ace", 13:"King", 12:"Queen", 11:"Jack", 10:"10", 9:"9", 8:"8", 7:"7", 6:"6", 5:"5", 4:"4", 3:"3", 2:"2"}
    
    def __eq__ (self, other):
        return (self.value == other.value)

    def __ne__ (self, other):
        return (self.value != other.value)

    def __lt__ (self, other):
        return (self.value < other.value)

    def __le__ (self, other):
        return (self.value <= other.value)

    def __gt__ (self, other):
        return (self.value > other.value)

    def __ge__ (self, other):
        return (self.value >= other.value)
    
    def __add__ (self, int):
        return card(self.suit, self.value+1)
    
    def __sub__ (self, int):
        return card(self.suit, self.value-1)
    
    def __repr__(self):
        return f'{self.values[self.value]} of {self.suit}'

class deck:
   
    def __init__(self, randomize=True):
        self.suits = {"D":"Diamonds", "C":"Clubs", "H": "Hearts", "S":"Spades"}
        self.values = {14:"Ace", 13:"King", 12:"Queen", 11:"Jack", 10:"10", 9:"9", 8:"8", 7:"7", 6:"6", 5:"5", 4:"4", 3:"3", 2:"2"}
        self.cards = np.array([])

        self.shuffle_deck(randomize)
    
    def shuffle_deck(self, randomize=True):
        self.cards = np.array([])
        for value in self.values.keys():
            for suit in self.suits.keys():
                self.cards = np.append(self.cards, card(suit, value))
        
        if randomize:
            np.random.shuffle(self.cards)

        return self.cards
    
    def __repr__(self):
        return f"{self.cards}"

    def draw(self):
        drawn_card = self.cards[0]
        self.cards = self.cards[1:]
        return drawn_card

class player:

    def __init__(self):
        self.hand = np.array([])
        self.money = 0
        self.status = True
        self.role = 4
        self.action = False
        self.high = None
        # {1: Dealer, 2: Little Blind, 3: Big Blind, 4: Nothing}
    
    def call(self, deck):
        self.money -= deck.call_amt

    def fold(self):
        self.status = False

    def raise_money(self, board, raise_amt):
        board.call_amt = raise_amt
        self.money -= board.call_amt


class board:

    def __init__(self, deck, blind_amt, players, buy_in, call_amt=0):
        self.cards = np.array([])
        self.pot = 0
        self.buy_in = buy_in
        self.call_amt = 0
        self.blind = blind_amt
        self.players = players
        self.deck = deck
    
    def show(self):
        if len(self.cards) == 0:
            num = 3
        else:
            num = 1    
        for card in range(num):
            self.cards = np.append(self.cards, self.deck.draw())
        return self.cards
    
    def deal_hands(self):
        for player in self.players:
            player.hand = np.append(player.hand, [self.deck.draw(), self.deck.draw()])
    
    def reset(self):
        for player in self.players:
            player.hand = []
        self.deck.shuffle_deck()
        self.cards = []
    
    def card_sort(self, cards):
        values_order = np.argsort(cards)[::-1]
        suits_order = np.argsort(np.array([card.suit for card in cards[values_order]]))
        
        return cards, values_order, suits_order
    
    def player_board(self, player):
        card_list = np.append(self.cards, player.hand)
        string_list = [f"{card.value}{card.suit}" for card in card_list]
        return 

    def isStraight(self, cards):
        cards, values_order, _ = self.card_sort(cards)
        
        same = 0
        for num in range(len(cards)-1):
            if cards[values_order][num] == cards[values_order][num+1]+1:
                same += 1
                if same >= 4:
                    return cards[values_order][num-3:num+2]
            else:
                same = 0
    
    def isStraight2(self, cards):
        cards, values_order, _ = self.card_sort(cards)

        in_row = cards[values_order][:-1] == cards[values_order][1:] + 1

    def isFlush(self, cards):
        cards, _, suits_order = self.card_sort(cards)
        
        same = 0
        for num in range(len(cards)-1):
            if cards[suits_order][num].suit == cards[suits_order][num+1].suit:
                same += 1
                if same >= 4:
                    return cards[suits_order][num-3:num+2]
            else:
                same = 0

    def isStraightFlush(self, cards):
        cards, values_order, suits_order = self.card_sort(cards)

        cards_in_order = np.array([])
        for num in range(len(cards)-1):
            if cards[values_order][num] == cards[values_order][num+1]+1:
                cards_in_order = np.append(cards_in_order, cards[values_order][num])
                print(cards_in_order)
                if np.size(cards_in_order) >= 4:
                    # Adds remaining cards if they are still consecutive
                    while num < len(cards)-1 and cards[values_order][num] == cards[values_order][num+1]+1 :
                        cards_in_order = np.append(cards_in_order, cards[values_order][num+1])
                        num += 1
                    break           
            else:
                cards_in_order = np.array([])
        return self.isFlush(cards_in_order)
    
    def isRoyalFlush(self, cards):
        if self.isStraightFlush(cards)[0].value == 14:
            return self.isStraightFlush(cards)

    def isFour(self, player):
        card_values, _ = self.player_board(player)
        high_single = 0
        same_count = 0

        for n in range(len(card_values)-1):
            if card_values[n] == card_values[n+1]:
                same_count += 1
                if same_count == 3:
                    #if four of a kind is the greatest four in array, the fifth in array is the high single
                    if high_single == 0:
                        high_single = card_values[n+2]
                    return True, (card_values[n], high_single)
            elif card_values[n] > high_single:
                high_single = card_values[n]
                same_count = 0
            else:
                same_count = 0
        return False, None
    
    def isTriple(self, player):
        card_values, _ = self.player_board(player)
        same_count = 0
        
        for n in range(len(card_values)-1):
            if card_values[n] == card_values[n+1]:
                same_count += 1
                if same_count == 2:
                    return True, card_values[n]
            else:
                same_count = 0
        return False, None
    
    def isDouble(self, player):
        card_values, _ = self.player_board(player)
        doubles = []
        
        for n in range(len(card_values)-1):
            if card_values[n] == card_values[n+1]:
                if same_count == 2:
                    return True, card_values[n]
            else:
                same_count = 0
        return False, None


In [2]:
play_deck = deck()

In [4]:
play_deck.cards

array([8 of D, 6 of H, 2 of S, Ace of S, 8 of C, 4 of H, 9 of C, 4 of D,
       5 of C, Jack of S, Jack of D, 4 of S, Jack of H, Queen of C,
       4 of C, 8 of H, 5 of S, Ace of H, 6 of D, Jack of C, 6 of S,
       King of S, 3 of S, 10 of D, 7 of D, Queen of H, 2 of C, 7 of S,
       Ace of C, 3 of D, King of C, 8 of S, Ace of D, Queen of D, 3 of C,
       10 of S, Queen of S, 5 of D, 2 of D, 2 of H, 9 of D, 9 of S,
       6 of C, King of H, 9 of H, 3 of H, 7 of C, King of D, 7 of H,
       10 of H, 10 of C, 5 of H], dtype=object)

In [456]:
player1 = player()
player2 = player()
player3 = player()
play_deck = deck(False)
game_board = board(play_deck, 10, [player1, player2, player3], 500)
game_board.deal_hands()

In [447]:
game_board.show()

array([King of Hearts, King of Spades, Queen of Diamonds, Queen of Clubs,
       Queen of Hearts], dtype=object)

In [448]:
player1.hand = play_deck.cards[0:17:4]
player1.hand

array([Queen of Spades, Jack of Spades, 10 of Spades, 9 of Spades,
       8 of Spades], dtype=object)

In [449]:
game_board.isStraightFlush(game_board.player_board(player1))

[King of Hearts]
[Queen of Diamonds]
[Queen of Diamonds Jack of Spades]
[Queen of Diamonds Jack of Spades 10 of Spades]
[Queen of Diamonds Jack of Spades 10 of Spades 9 of Spades]


In [424]:
game_board.player_board(player1)

array([King of Hearts, Queen of Spades, 10 of Diamonds, 9 of Clubs],
      dtype=object)

In [383]:
c[v]

array([King of Spades, 8 of Spades, 7 of Clubs, 5 of Hearts,
       3 of Diamonds], dtype=object)

In [377]:
v

array([-2, -1, -4,  0, -3])

In [418]:
if function():
    print("hi")

run


In [263]:
array = np.array([1, 2, 3 ,4, 5])

In [417]:
def function():
    print("run")
    return None

def function2():
    print("non")

In [287]:
if 2:
    print("hi")

hi


In [299]:
np.sort([card("Diamond", 4), card("Club", 3)])

array([3 of Club, 4 of Diamond], dtype=object)

In [457]:
card_list = play_deck.cards[0:17:2]
card_list

array([King of H, Queen of D, Queen of H, Jack of D, Jack of H, 10 of D,
       10 of H, 9 of D, 9 of H], dtype=object)

In [481]:
string_list = np.array([[card.value for card in card_list],[card.suit for card in card_list]])
string_list[1, :]

array(['H', 'D', 'H', 'D', 'H', 'D', 'H', 'D', 'H'], dtype='<U21')

In [476]:
int(string_list[0,:])

TypeError: only size-1 arrays can be converted to Python scalars

In [477]:
string_list[:][0].astype(int)

array([13, 12, 12, 11, 11, 10, 10,  9,  9])

In [464]:
A = np.array([[card.value for card in card_list],[card.suit for card in card_list]])

In [475]:
[card.value for card in card_list]

[13, 12, 12, 11, 11, 10, 10, 9, 9]

In [4]:
list1 = [1,2,3,4]

In [18]:
var = 1

In [6]:
print(var in list1)

True


In [24]:
var = var+1 if var+1 != len(list1) else 0
var

3

In [27]:
multiplier = 3 if var == 7 else 8
multiplier

3

In [26]:
var = 7